# AgentsVille Trip Planner - Project Assignment

In this project, you'll implement an AI system to help you plan a trip--to the wonderful city of AgentsVille!

Your AI system will demonstrate advanced LLM reasoning techniques including:

1. **Role-Based Prompting** - Your agent will act as a specialized travel planner
2. **Chain-of-Thought Reasoning** - Step-by-step planning of itineraries
3. **ReAct Prompting** - Thought → Action → Observation cycles
4. **Feedback Loops** - Self-evaluation using tools in the ReAct loop to find mistakes and improve plans

You'll simulate external API calls to gather weather data and activities. Then, process this information to create personalized travel itineraries based on interests and other constraints. Last, you'll implement a feedback loop to refine the plan.

Your task is to build a travel agent that can plan the perfect AgentsVille vacation!

## Project Instructions

Here are the steps you'll follow:

1. **Define Vacation Details**
    - Specify the trip duration, interests, and constraints.
    - Use Pydantic to structure and verify this information in a class called `VacationInfo`.
2. **Review Weather and Activity schedules**
    - Simulate API calls to gather weather data and available activities in bulk
    - Review the data manually to understand the available options
3. **The ItineraryAgent**
    - Implement an agent that generates a day-by-day itinerary based on the vacation details
    - The system prompt will guide the agent's reasoning through a step-by-step planning process to take travel preferences (e.g. destination, dates, interests) and generate a detailed day-by-day itinerary
    - Craft the components of the prompt (including the role, task/instructions, output format, examples, and context) to elicit the best possible itinerary in one LLM call
4. **Evaluating the Itinerary**
    - Evaluate the itinerary using a set of criteria to ensure a high-quality travel plan
        - For instance, does the itinerary match the city and the dates requested?
        - Or, is the total cost calulation accurate and is it within budget?
        - Or, does the agent hallucinate any activities that are not available?
        - Or, does the agent suggest activities that are not suitable for the weather? This specific evaluation function will require the use of an LLM to compare the event description against the weather data.
5. **Defining the Tools**
    - We will define four tools to assist the agent
        - **calculator_tool**: to accurately calculate costs
        - **get_activities_by_date_tool**: to retrieve activities for a specific date
        - **run_evals_tool**: to evaluate the itinerary against the criteria
        - **final_answer_tool**: to provide the final answer in a structured format
6. **The ItineraryRevisionAgent**
    - We will implement a second agent that revises the itinerary based on feedback using the ReAct THOUGHT → ACTION → OBSERVATION cycle.
        - The LLM will first generated a THOUGHT / ACTION message, which contains reasoning steps and a tool call invocation.
        - The Python code will parse the tool call and execute it, returning the result as a string to the LLM in an OBSERVATION message.
        - After this cycle repeats n number of times, the LLM will invoke the final_answer_tool to signal to the Python code to end the loop and return the final answer.
    - This agent will also **incorporate feedback on the initial itinerary** from the travelers to ensure the final plan has **at least 2 activities per day**. A new evaluation function using a powerful LLM will be created to check this user feedback.
    - The agent will use the tools above to refine the plan iteratively, checking the weather and available activities, and ensuring the itinerary meets all constraints.
7. **Something just for fun!**
    - To wrap things up we'll create a fun, narrative summary of the trip, highlighting the best activities and experiences!

## Initial Setup

Let's start with settin up our environment and defining the vacation details.

In [4]:
# When using VSCode in the Udacity workspace, add /workspace to the PYTHON_PATH
import os
import sys

WORKSPACE_DIRECTORY = "/workspace"
if os.path.exists(WORKSPACE_DIRECTORY) and WORKSPACE_DIRECTORY not in sys.path:
    sys.path.append(WORKSPACE_DIRECTORY)
    print(f"Added {WORKSPACE_DIRECTORY} to the Python path")

In [5]:
# Install required packages if not already installed
# No changes needed here.
%pip install -q json-repair==0.47.1 numexpr==2.11.0 openai==1.74.0 pandas==2.3.0 pydantic==2.11.7 python-dotenv==1.1.0


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(dotenv_path='../../.env')

client = OpenAI(
    # Change the base_url when using the Vocareum API endpoint
    # If using the OpenAI API endpoint, you can comment out the base_url line
    base_url="https://openai.vocareum.com/v1",
    # Uncomment one of the following
    # api_key="**********",  # <--- TODO: Fill in your Vocareum API key here
    api_key=os.getenv(
        "OPENAI_API_KEY"
    ),  # <-- Load from .env file
)


In [7]:
# Throughout this project you can experiment with different OpenAI models.
# By default we will use GPT-4.1-mini, which is a good balance of speed and cost.
from enum import Enum

class OpenAIModel(str, Enum):
    GPT_41 = "gpt-4.1"  # Strong default choice for development tasks, particularly those requiring speed, responsiveness, and general-purpose reasoning. 
    GPT_41_MINI = "gpt-4.1-mini"  # Fast and affordable, good for brainstorming, drafting, and tasks that don't require the full power of GPT-4.1.
    GPT_41_NANO = "gpt-4.1-nano"  # The fastest and cheapest model, suitable for lightweight tasks, high-frequency usage, and edge computing.

MODEL = OpenAIModel.GPT_41_MINI  # Default model for this project


## Define Vacation Details

Let's encode the details of our vacation in JSON format and verify it using Pydantic.

In practice, a chatbot agent could gather the information of a user. After it has gathered all the information it needs, it could generate this JSON object from the chat transcript. We will skip that step to focus on the itinerary generation itself.

In [8]:
# The Vacation Info Data Structure
# No changes needed here, but you may choose to personalize the data.

VACATION_INFO_DICT = {
    "travelers": [
        {
            "name": "Tyler",
            "age": 33,
            # Possible interests: art, cooking, comedy, dancing, fitness, gardening, hiking, movies,
            # music, photography, reading, sports, technology, theatre, tennis, writing
            "interests": ["music", "cooking", "comedy", "technology"],
        },
        {
            "name": "Annie",
            "age": 25,
            # Possible interests: art, cooking, comedy, dancing, fitness, gardening, hiking, movies,
            # music, photography, reading, sports, technology, theatre, tennis, writing
            "interests": ["fitness", "music", "theatre", "art"],
        },
    ],
    "destination": "AgentsVille",
    "date_of_arrival": "2025-06-10",  # Mock data exists for 2025-06-10
    "date_of_departure": "2025-06-12",  # ...until 2025-06-15.
    "budget": 130,  # Budget is in fictional currency units.
}

In [9]:
import importlib
import utils
import models

# Reload modules to ensure we have the latest Interest enum
importlib.reload(utils)
importlib.reload(models)

from utils import Interest
from typing import List
import datetime
from pprint import pprint
from models import Traveler, VacationInfo

# Validate the VacationInfo data structure
vacation_info = VacationInfo.model_validate(VACATION_INFO_DICT)

# Display the vacation info as a dictionary
pprint(vacation_info.model_dump())

# Check that VacationInfo contains the expected data
assert "travelers" in vacation_info.model_dump().keys(), "VacationInfo should contain 'travelers' key"
assert "destination" in vacation_info.model_dump().keys(), "VacationInfo should contain 'destination' key"
assert "date_of_arrival" in vacation_info.model_dump().keys(), "VacationInfo should contain 'date_of_arrival' key"
assert "date_of_departure" in vacation_info.model_dump().keys(), "VacationInfo should contain 'date_of_departure' key"
assert "budget" in vacation_info.model_dump().keys(), "VacationInfo should contain 'budget' key"
assert isinstance(vacation_info.travelers, list), "Travelers should be a list"
assert all(isinstance(traveler, Traveler) for traveler in vacation_info.travelers), "All travelers should be instances of Traveler class"
assert isinstance(vacation_info.date_of_arrival, datetime.date), "date_of_arrival should be a date"
assert isinstance(vacation_info.date_of_departure, datetime.date), "date_of_departure should be a date"
assert isinstance(vacation_info.budget, int), "budget should be an integer"

# If all assertions pass, print a success message
print("✅ VacationInfo data structure is valid!")

{'budget': 130,
 'date_of_arrival': datetime.date(2025, 6, 10),
 'date_of_departure': datetime.date(2025, 6, 12),
 'destination': 'AgentsVille',
 'travelers': [{'age': 33,
                'interests': [<Interest.MUSIC: 'music'>,
                              <Interest.COOKING: 'cooking'>,
                              <Interest.COMEDY: 'comedy'>,
                              <Interest.TECHNOLOGY: 'technology'>],
                'name': 'Tyler'},
               {'age': 25,
                'interests': [<Interest.FITNESS: 'fitness'>,
                              <Interest.MUSIC: 'music'>,
                              <Interest.THEATRE: 'theatre'>,
                              <Interest.ART: 'art'>],
                'name': 'Annie'}]}
✅ VacationInfo data structure is valid!


## Review Weather and Activity Schedules

Now that we have the trip details, we can retrieve the weather and activity schedules for the dates of the trip.

We will call an API to get all the data at once, in order to be able to include it in the context for our itinerary planning agent.

Also, we will format the data as Pandas DataFrames for easier viewing. Take the time to read and understand the data to see if the agent notices the same things you do. For instance:
- What does the weather look like for the trip? On what days it is sunny, rainy, or cloudy?
- What activities are available on each day? Are there any special events or festivals related to the user's interests?

In [10]:
# The `call_weather_api_mocked` mocks calling a weather API to get weather data


from utils import call_weather_api_mocked
import pandas as pd

pd.set_option("display.max_colwidth", None)  # Show full content in DataFrame cells

weather_for_dates = [
    call_weather_api_mocked(
        date=ts.strftime("%Y-%m-%d"), city=vacation_info.destination
    )
    for ts in pd.date_range(
        start=vacation_info.date_of_arrival,
        end=vacation_info.date_of_departure,
        freq="D",
    )
]

weather_for_dates_df = pd.DataFrame(weather_for_dates)

weather_for_dates_df

,date,city,temperature,temperature_unit,condition,description
0,2025-06-10,AgentsVille,31,celsius,clear,A bright and sunny day in AgentsVille with clear skies and warm temperatures. Perfect weather for outdoor activities!
1,2025-06-11,AgentsVille,34,celsius,partly cloudy,"A warm day with periods of sunshine and mixed clouds, making it a perfect opportunity for outdoor activities."
2,2025-06-12,AgentsVille,28,celsius,thunderstorm,"A thunderstorm is expected to roll in during the afternoon, bringing heavy rain and gusty winds. The atmosphere will feel charged with humidity, creating a sultry and dramatic setting as clouds build in the sky."


In [11]:
# The `call_activities_api_mocked` function returns the activities for a given date and city.

from utils import call_activities_api_mocked

activities_for_dates = [
    activity
    for ts in pd.date_range(
        start=vacation_info.date_of_arrival,
        end=vacation_info.date_of_departure,
        freq="D",
    )
    for activity in call_activities_api_mocked(
        date=ts.strftime("%Y-%m-%d"), city=vacation_info.destination
    )
]

activities_for_dates_df = pd.DataFrame(activities_for_dates)

activities_for_dates_df

,activity_id,name,start_time,end_time,location,description,price,related_interests
0,event-2025-06-10-0,FutureTech Breakfast Meet-Up,2025-06-10 09:00,2025-06-10 11:00,"The Innovation Atrium, Tech District, AgentsVille","Join fellow technology enthusiasts for a dynamic morning at the FutureTech Breakfast Meet-Up! Dive into the latest trends in tech, gadget demos, and networking opportunities over coffee and fresh pastries. Held indoors at the spacious Innovation Atrium, this event is perfect for tech lovers eager to exchange ideas and discover new possibilities in a comfortable, modern setting.",20,[technology]
1,event-2025-06-10-1,Serve & Savor: Tennis and Taste Luncheon,2025-06-10 12:00,2025-06-10 13:30,"The Grand Racquet Terrace, AgentsVille","Join us for 'Serve & Savor,' the ultimate crossover event for cooking and tennis enthusiasts in AgentsVille! Kick off your lunch hour with a friendly round of doubles on our outdoor courts, then unwind with a hands-on cooking workshop led by a local chef, where you'll prepare and enjoy delicious energy-boosting recipes. Whether you come for the sport or the flavors, this energizing luncheon celebrates both passions in a lively outdoor setting. Ideal for anyone who loves to play, cook, or simply savor fresh food and fun!",20,"[cooking, tennis]"
2,event-2025-06-10-2,Artful Athletics: Paint & Play Extravaganza,2025-06-10 15:00,2025-06-10 17:00,"Creative Courts Park, AgentsVille","Join us for an exciting afternoon at Creative Courts Park, where the worlds of art and sports collide! At 'Artful Athletics: Paint & Play Extravaganza', you'll participate in collaborative outdoor murals inspired by your favorite sports, and then get moving with fun, friendly sports mini-games. Whether you love painting or playing, this event celebrates creativity, teamwork, and the joy of movement under the open sky. Perfect for art lovers and sports enthusiasts alike—come ready to express yourself and get active! (Event is held outdoors; in case of rain, we move indoors to the Community Gym nearby.)",15,"[art, sports]"
3,event-2025-06-10-3,AgentsVille Twilight Writing Escape,2025-06-10 19:00,2025-06-10 21:00,"The Ink Loft, 12 Quill Lane, AgentsVille","Join fellow writers for an inspiring evening at The Ink Loft, where words flow as freely as the coffee! This writing-themed event welcomes all—novelists, poets, bloggers, or anyone with a passion for storytelling. Set indoors in AgentsVille's coziest lounge, enjoy writing games, group prompts, and opportunities to read your work aloud. Connect, create, and celebrate the art of writing in this creative indoor haven.",15,"[writing, reading, art]"
4,event-2025-06-11-0,Morning Groove Dance Party,2025-06-11 09:00,2025-06-11 10:30,"Rhythm Hall, Center Plaza, AgentsVille","Start your day with energy and joy at the Morning Groove Dance Party! This lively event welcomes dancers of all levels to join a vibrant indoor session filled with upbeat music and fun routines. Whether you love modern pop, Latin beats, or classic disco, our dance instructors will guide you to move and groove. Connect with fellow dance lovers in the colorful atmosphere of Rhythm Hall. Perfect for fans of dancing, music, and fitness. Let the rhythm move you! (Indoor event.)",15,"[dancing, music, fitness]"
5,event-2025-06-11-1,Tech Lunch & Learn: AI Frontiers,2025-06-11 12:00,2025-06-11 13:30,"The Digital Atrium, AgentsVille","Join fellow tech enthusiasts for a dynamic lunchtime event exploring the future of artificial intelligence! Held indoors at The Digital Atrium, this Tech Lunch & Learn features engaging lightning talks, interactive demos, and networking opportunities all centered around technology and innovation. Enjoy light lunch fare as you connect with others passionate about technology, AI, and the digital world. Whether you're a seasoned developer or just curious about tech, this event is for you! Related interests: technology, music (sound tech demos), photography (AI imaging), writing (AI cr

## The ItineraryAgent (ITINERARY_AGENT_SYSTEM_PROMPT)

First we will review the Pydantic objects used for defining the output of our agent, the TravelPlan, ItineraryDay, Activity, and Weather classes.

Second, we will create a Chain-of-Thought prompt to guide the agent in planning the trip. This prompt will instruct the agent to consider the weather, activities, and user preferences when creating the itinerary.

Third, we will run the agent to produce the TravelPlan object, which will will refine in the following steps.

In [13]:
import json 
from utils import ChatAgent
from typing import Optional
from models import TravelPlan, VacationInfo

ITINERARY_AGENT_SYSTEM_PROMPT = f"""
You are a Itinerary Planning Agent.

## Task
Plan a itinerary for the following travelers:

Note that, outdoor-only activities should be avoided
during rain, events should be chosen based on traveler interests,
and the budget should not be exceeded, and there should be at least
one activity per day. Make sure that there is no overlap in the activities.

## Output Format

Respond using two sections (ANALYSIS AND FINAL OUTPUT) in the following format:

    ANALYSIS:
    * step-by-step analysis


    FINAL OUTPUT:

    ```json
    {TravelPlan.model_json_schema()}
    ```

## Context

Make sure to only use the following data to plan. DO NOT make up any information or create new data. Make sure the event Ids line up with the activities in the activities_for_dates_df.

Weather Data: {weather_for_dates_df}
Activities availible for the dates: {activities_for_dates_df}

Neccessary user information will be passed in the user prompt.

## Example Output
ANALYSIS:
* The weather is clear on June 10 and partly cloudy on June 11, allowing for both indoor and outdoor activities.
* Selected activities are matched to the traveler's interests and budget.
* No overlapping activities, and at least one activity per day is scheduled.

FINAL OUTPUT:

```json
{"traveler_name": "Alex", "start_date": "2025-06-10", "end_date": "2025-06-11", "total_cost": 35, "itinerary": [{"date": "2025-06-10", "activities": [{"activity_id": "event-2025-06-10-0", "name": "FutureTech Breakfast Meet-Up", "start_time": "09:00", "end_time": "11:00"}]}, {"date": "2025-06-11", "activities": [{"activity_id": "event-2025-06-11-1", "name": "Tech Lunch & Learn", "start_time": "12:00", "end_time": "13:00"}]}]}
```

"""  # noqa


assert "TASK" in ITINERARY_AGENT_SYSTEM_PROMPT.upper(), "❌ ITINERARY_AGENT_SYSTEM_PROMPT should contain a 'TASK' section"
assert "OUTPUT FORMAT" in ITINERARY_AGENT_SYSTEM_PROMPT.upper(), "❌ ITINERARY_AGENT_SYSTEM_PROMPT should contain a 'OUTPUT FORMAT' section"


class ItineraryAgent(ChatAgent):
    """An agent that plans itineraries based on vacation information, weather, and activities."""
    system_prompt = ITINERARY_AGENT_SYSTEM_PROMPT

    def get_itinerary(self, vacation_info: VacationInfo, model: Optional[OpenAIModel] = None) -> TravelPlan:
        """Generates a travel itinerary based on the provided vacation information."""
        from utils import print_in_box
        response = (self.chat(
            user_message=vacation_info.model_dump_json(indent=2),
            add_to_messages=False,
            model=model or self.model,
        ) or "").strip()

        print_in_box(response, "Raw Response")

        # Parse the response
        json_text = response.strip()

        if "```json" in json_text:
            json_text = json_text.split("```json")[1].split("```")[0].strip()

        try:
            travel_plan = TravelPlan.model_validate_json(json_text)
            return travel_plan
        except Exception as e:
            print("Error validating the following text as TravelPlan JSON:")
            print(json_text)
            raise

itinerary_agent = ItineraryAgent(client=client, model=MODEL)

SyntaxError: f-string: expressions nested too deeply (2058492522.py, line 52)

In [10]:
# Generate the travel itinerary
# No changes needed here, though you can change the model to a different one if you want.

travel_plan_1 = itinerary_agent.get_itinerary(
    vacation_info=vacation_info,
    model=MODEL,  # Optionally, you can change the model here
)

if travel_plan_1 is not None:
    print("✅ Initial itinerary generated successfully. Congratulations!")


╔═══════════════════════════════════════════[ ItineraryAgent - User Prompt ]═══════════════════════════════════════════╗
║ {                                                                                                                    ║
║   "travelers": [                                                                                                     ║
║     {                                                                                                                ║
║       "name": "Tyler",                                                                                               ║
║       "age": 33,                                                                                                     ║
║       "interests": [                                                                                                 ║
║         "music",                                                                                                     ║
║         "cooking",           

In [11]:
pprint(travel_plan_1.model_dump_json())

('{"city":"AgentsVille","start_date":"2025-06-10","end_date":"2025-06-12","total_cost":75,"itinerary_days":[{"date":"2025-06-10","weather":{"temperature":31.0,"temperature_unit":"celsius","condition":"clear"},"activity_recommendations":[{"activity":{"activity_id":"event-2025-06-10-1","name":"Serve '
 '& Savor: Tennis and Taste '
 'Luncheon","start_time":"2025-06-10T12:00:00","end_time":"2025-06-10T13:30:00","location":"The '
 'Grand Racquet Terrace, AgentsVille","description":"Join us for \'Serve & '
 "Savor,' the ultimate crossover event for cooking and tennis enthusiasts in "
 'AgentsVille! Kick off your lunch hour with a friendly round of doubles on '
 'our outdoor courts, then unwind with a hands-on cooking workshop led by a '
 "local chef, where you'll prepare and enjoy delicious energy-boosting "
 'recipes. Whether you come for the sport or the flavors, this energizing '
 'luncheon celebrates both passions in a lively outdoor setting. Ideal for '
 'anyone who loves to play, cook,

## Evaluating the Itinerary

We've successfully created an itinerary, but how do we know if it's any good?

Now we will create some evaluation functions (sometimes called evals) to help us determine the quality of the itinerary. We will not only want our final output to be of the highest quality possible initially, but we also want to give the chance for the LLM to reflect on its own output and make improvements at a second pass.

If the itinerary does not meet all the criteria specified here, no worries! Afterwards, we will implement a feedback loop that will allow the agent to improve its plan iteratively.

In [12]:
from test import (
    AgentError,
    get_eval_results,
    eval_start_end_dates_match,
    eval_total_cost_is_accurate,
    eval_total_cost_is_within_budget,
    eval_itinerary_events_match_actual_events,
    eval_itinerary_satisfies_interests
)

## ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT

In [ ]:
# Use an LLM to determine whether an event should be avoided due to weather conditions.

ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT = """
You are a weather expert.

## Task
Note: When there is not enough information, assume the activity IS_COMPATIBLE
with the weather. Also, look out for backup options mentioned in the activity description.

## Output format

    REASONING:
    * use step by step reasoning to justify your answer.

    FINAL ANSWER:
    [IS_COMPATIBLE, IS_INCOMPATIBLE]

## Examples
`
## Example 1: IS_COMPATIBLE

Activity: "Morning Groove Dance Party"
Description: "A high-energy dance party held in a spacious indoor ballroom with music and refreshments."
Weather Condition: "thunderstorm"

REASONING:
* The event is hosted indoors, so participants will not be affected by outdoor conditions, including thunderstorms.
* There is no requirement for the activity to be outside, and the description specifies an indoor setting.

FINAL ANSWER:
IS_COMPATIBLE

---

## Example 2: IS_INCOMPATIBLE

Activity: "AgentsVille Outdoor Tennis Tournament"
Description: "A competitive tennis tournament held outdoors on the city courts; event will be cancelled in case of rain."
Weather Condition: "rain"

REASONING:
* The tournament is explicitly held outdoors.
* The description mentions cancellation in case of rain, indicating the event cannot proceed during rainy conditions.

FINAL ANSWER:
IS_INCOMPATIBLE


""".strip()


# Defined here for clarity
def eval_activities_and_weather_are_compatible(
    vacation_info: VacationInfo, final_output: TravelPlan, system_prompt: str = None
):
    """Verifies that no outdoor-only activities are scheduled during inclement weather conditions.

    Args:
        vacation_info (dict): Contains the vacation details
        final_output (dict): Contains the itinerary details including daily activities and weather conditions
        system_prompt (str, optional): The system prompt to use for compatibility checking. 
                                       Defaults to ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT if not provided.

    Raises:
        AgentError: If any outdoor activities are scheduled during weather conditions that could ruin them
    """
    from utils import do_chat_completion
    
    # Use provided prompt or fall back to global variable
    prompt = system_prompt if system_prompt is not None else ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT

    activities_that_are_incompatible = []

    for itinerary_day in final_output.itinerary_days:
        weather_condition = itinerary_day.weather.condition

        for activity_recommendation in itinerary_day.activity_recommendations:
            resp = do_chat_completion(
                messages=[
                    {
                        "role": "system",
                        "content": prompt,
                    },
                    {
                        "role": "user",
                        "content": f"Activity: {activity_recommendation.activity.name}\nDescription: {activity_recommendation.activity.description}\nWeather Condition: {weather_condition}",
                    },
                ],
                client=client,
                # This is a high-frequency use case, so we use a fast and cheap model.
                model=OpenAIModel.GPT_41_NANO,
            )
    


            if "IS_COMPATIBLE" in (resp or ""):
                is_compatible = True
            elif "IS_INCOMPATIBLE" in (resp or ""):
                is_compatible = False
            else:
                raise RuntimeError(
                    f"Unexpected response from the model: {resp}. Expected 'IS_COMPATIBLE' or 'IS_INCOMPATIBLE'."
                )

            if is_compatible:
                print(
                    f"✅ Activity {activity_recommendation.activity.name} (on {itinerary_day.date}) and weather '{weather_condition}' are compatible."
                )

            else:
                activities_that_are_incompatible.append(
                    activity_recommendation.activity.name
                )
                print(
                    f"❌ Activity {activity_recommendation.activity.name} (on {itinerary_day.date}) and weather '{weather_condition}' are incompatible."
                )

    if activities_that_are_incompatible:
        raise AgentError(
            f"Activities that may be ruined by inclement weather: {activities_that_are_incompatible}"
        )


In [14]:
# Run all of the evaluation functions again
# No changes needed here.

ALL_EVAL_FUNCTIONS = [
    eval_start_end_dates_match,
    eval_total_cost_is_accurate,
    eval_itinerary_events_match_actual_events,
    eval_itinerary_satisfies_interests,
    eval_total_cost_is_within_budget,
    eval_activities_and_weather_are_compatible,
]

eval_results = get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=ALL_EVAL_FUNCTIONS,
)

eval_results.model_dump()

✅ Traveler Tyler has a match with interest {<Interest.COOKING: 'cooking'>} at Serve & Savor: Tennis and Taste Luncheon
✅ Traveler Tyler has a match with interest {<Interest.MUSIC: 'music'>} at Morning Groove Dance Party
✅ Traveler Tyler has a match with interest {<Interest.COOKING: 'cooking'>} at Palette & Palate: Art Meets Cooking Experience
✅ Traveler Tyler has a match with interest {<Interest.TECHNOLOGY: 'technology'>} at Tech & Film Fusion Night
✅ Traveler Annie has a match with interest {<Interest.MUSIC: 'music'>, <Interest.FITNESS: 'fitness'>} at Morning Groove Dance Party
✅ Traveler Annie has a match with interest {<Interest.ART: 'art'>} at Palette & Palate: Art Meets Cooking Experience
✅ Activity Serve & Savor: Tennis and Taste Luncheon (on 2025-06-10) and weather 'clear' are compatible.
✅ Activity Morning Groove Dance Party (on 2025-06-11) and weather 'partly cloudy' are compatible.
✅ Activity Palette & Palate: Art Meets Cooking Experience (on 2025-06-11) and weather 'partly c

{'success': True,
 'failures': [],
 'eval_functions': ['eval_start_end_dates_match',
  'eval_total_cost_is_accurate',
  'eval_itinerary_events_match_actual_events',
  'eval_itinerary_satisfies_interests',
  'eval_total_cost_is_within_budget',
  'eval_activities_and_weather_are_compatible']}

## Defining the Tools

Our ItineraryRevisionAgent will be a ReAct-based agent that will use tools to:
- Evaluate/Re-evaluate the itinerary
- Use a calculator since LLMs sometimes struggle with arithmetic
- Call the activities API to get more information about activities
- Return the final itinerary


In [15]:
# Helper function to generate tool descriptions from function docstrings
# No changes needed here.

def get_tool_descriptions_string(fns):
    """Generates a tool description from a function's docstring.
    Args:
        fns (list): List of functions to generate descriptions for.
    Returns:
        str: A formatted string containing the function names and their descriptions."""
    resp = ""
    for fn in fns:
        function_name = fn.__name__
        function_doc = fn.__doc__ or "No description provided."

        resp += f"* `{function_name}`: {function_doc}\n"

    return resp

In [16]:
# Define the calculator tool that evaluates mathematical expressions.
# No changes needed here.

def calculator_tool(input_expression) -> float:
    """Evaluates a mathematical expression and returns the result as a float.

    Args:
        input_expression (str): A string containing a valid mathematical expression to evaluate.

    Returns:
        float: The result of the evaluated expression.

    Example:
        >>> calculator_tool("1 + 1")
        2.0
    """
    import numexpr as ne
    return float(ne.evaluate(input_expression))


assert calculator_tool("1 + 1") == 2.0

print(get_tool_descriptions_string([calculator_tool]))

* `calculator_tool`: Evaluates a mathematical expression and returns the result as a float.

    Args:
        input_expression (str): A string containing a valid mathematical expression to evaluate.

    Returns:
        float: The result of the evaluated expression.

    Example:
        >>> calculator_tool("1 + 1")
        2.0
    



In [17]:
from models import Activity

def get_activities_by_date_tool(date: str, city: str) -> List[dict]:
    """
    Retrieves a list of activity dictionaries for a specified date and city.

    Args:
        date (str): The date (YYYY-MM-DD format) for which to fetch activities.
        city (str): The city for which to fetch available activities.

    Returns:
        List[dict]: A list of dictionaries, each representing an activity occurring on the given date in the specified city.

    Example:
        >>> get_activities_by_date_tool("2025-06-10", "AgentsVille")
        [{'activity_id': 'event-2025-06-10-0', 'name': 'FutureTech Breakfast Meet-Up', ...}, ...]
    """
    from utils import call_activities_api_mocked
    resp = call_activities_api_mocked(date=date, city=city)

    return [Activity.model_validate(activity).model_dump() for activity in resp]



assert len(get_activities_by_date_tool("2025-06-10", "AgentsVille")) > 0

print(get_tool_descriptions_string([get_activities_by_date_tool]))

* `get_activities_by_date_tool`: 
    Retrieves a list of activity dictionaries for a specified date and city.

    Args:
        date (str): The date (YYYY-MM-DD format) for which to fetch activities.
        city (str): The city for which to fetch available activities.

    Returns:
        List[dict]: A list of dictionaries, each representing an activity occurring on the given date in the specified city.

    Example:
        >>> get_activities_by_date_tool("2025-06-10", "AgentsVille")
        [{'activity_id': 'event-2025-06-10-0', 'name': 'FutureTech Breakfast Meet-Up', ...}, ...]
    



In [18]:
def run_evals_tool(travel_plan: TravelPlan) -> dict:
    """Runs all evaluation tools on the provided travel plan and vacation info.

    Args:
        travel_plan (TravelPlan): The travel plan to evaluate.

    Returns:
        EvaluationResults: The results of the evaluations.
    """
    if isinstance(travel_plan, dict):
        travel_plan = TravelPlan.model_validate(travel_plan)

    resp = get_eval_results(
        vacation_info=vacation_info,
        final_output=travel_plan,
        eval_functions=ALL_EVAL_FUNCTIONS,
    )
    return {
        # Show the success status and any failures
        "success": resp.success,
        "failures": resp.failures,
    }


print(get_tool_descriptions_string([run_evals_tool]))

* `run_evals_tool`: Runs all evaluation tools on the provided travel plan and vacation info.

    Args:
        travel_plan (TravelPlan): The travel plan to evaluate.

    Returns:
        EvaluationResults: The results of the evaluations.
    



In [19]:
# Let's double check that the tool works as expected.
# You should see the same results as before
run_evals_tool(travel_plan=travel_plan_1)

✅ Traveler Tyler has a match with interest {<Interest.COOKING: 'cooking'>} at Serve & Savor: Tennis and Taste Luncheon
✅ Traveler Tyler has a match with interest {<Interest.MUSIC: 'music'>} at Morning Groove Dance Party
✅ Traveler Tyler has a match with interest {<Interest.COOKING: 'cooking'>} at Palette & Palate: Art Meets Cooking Experience
✅ Traveler Tyler has a match with interest {<Interest.TECHNOLOGY: 'technology'>} at Tech & Film Fusion Night
✅ Traveler Annie has a match with interest {<Interest.MUSIC: 'music'>, <Interest.FITNESS: 'fitness'>} at Morning Groove Dance Party
✅ Traveler Annie has a match with interest {<Interest.ART: 'art'>} at Palette & Palate: Art Meets Cooking Experience
✅ Activity Serve & Savor: Tennis and Taste Luncheon (on 2025-06-10) and weather 'clear' are compatible.
✅ Activity Morning Groove Dance Party (on 2025-06-11) and weather 'partly cloudy' are compatible.
✅ Activity Palette & Palate: Art Meets Cooking Experience (on 2025-06-11) and weather 'partly c

{'success': True, 'failures': []}

In [20]:
# A tool to return the final travel plan
# No changes needed here.

def final_answer_tool(final_output: TravelPlan) -> TravelPlan:
    """Returns the final travel plan

    Args:
        final_output (TravelPlan): The final travel plan to return.

    Returns:
        TravelPlan: The final travel plan.
    """
    return final_output


print(get_tool_descriptions_string([final_answer_tool]))

* `final_answer_tool`: Returns the final travel plan

    Args:
        final_output (TravelPlan): The final travel plan to return.

    Returns:
        TravelPlan: The final travel plan.
    



In [21]:
# List of all tools available for the agent
# No changes needed here.

ALL_TOOLS = [
    calculator_tool,
    get_activities_by_date_tool,
    run_evals_tool,
    final_answer_tool,
]
print(get_tool_descriptions_string(ALL_TOOLS))

* `calculator_tool`: Evaluates a mathematical expression and returns the result as a float.

    Args:
        input_expression (str): A string containing a valid mathematical expression to evaluate.

    Returns:
        float: The result of the evaluated expression.

    Example:
        >>> calculator_tool("1 + 1")
        2.0
    
* `get_activities_by_date_tool`: 
    Retrieves a list of activity dictionaries for a specified date and city.

    Args:
        date (str): The date (YYYY-MM-DD format) for which to fetch activities.
        city (str): The city for which to fetch available activities.

    Returns:
        List[dict]: A list of dictionaries, each representing an activity occurring on the given date in the specified city.

    Example:
        >>> get_activities_by_date_tool("2025-06-10", "AgentsVille")
        [{'activity_id': 'event-2025-06-10-0', 'name': 'FutureTech Breakfast Meet-Up', ...}, ...]
    
* `run_evals_tool`: Runs all evaluation tools on the provided trav

## The ItineraryRevisionAgent

The ItineraryRevisionAgent will
* take initial feedback from the user about the itinerary and
* use the tools defined above

to refine original itinerary iteratively using a ReAct-based approach.

In [22]:
# Get the traveler's feedback and create a new evaluation function to check if the feedback was incorporated.
# No changes needed here.

TRAVELER_FEEDBACK = "I want to have at least two activities per day."


def eval_traveler_feedback_is_incorporated(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Checks if the traveler's feedback was incorporated into the revised travel plan.

    Args:
        vacation_info (VacationInfo): The vacation information.
        final_output (TravelPlan): The revised travel plan.

    Raises:
        AgentError: If the traveler's feedback was not successfully incorporated.
    """

    agent = ChatAgent(
        system_prompt="""You are an expert in evaluating whether a travel plan incorporates traveler feedback.

    ## Output Format

    Respond using two sections (ANALYSIS AND FINAL OUTPUT) in the following format:

        ANALYSIS:
        * [step-by-step analysis]


        FINAL OUTPUT:
        [FULLY_INCORPORATED, PARTIALLY_INCORPORATED, NOT_INCORPORATED, or UNKNOWN]
        REASON: [reasoning for the final output]

    """,
        client=client,
        model=OpenAIModel.GPT_41,  # Use a powerful model for checking traveler feedback
    )

    resp = agent.chat(
        f"""Traveler Feedback: {TRAVELER_FEEDBACK}
    Revised Travel Plan: {final_output.model_dump_json()}
    """,
    )
    if "FINAL OUTPUT:" not in resp:
        raise RuntimeError(
            f"Unexpected response from the model: {resp}. Expected 'FINAL OUTPUT:'."
        )
    if "FULLY_INCORPORATED" not in resp:
        final_output = resp.split("FINAL OUTPUT:")[-1].strip()
        raise AgentError(
            f"Traveler feedback was not successfully incorporated into the revised travel plan. Response: {final_output}"
        )

ALL_EVAL_FUNCTIONS = [
    eval_start_end_dates_match,
    eval_total_cost_is_accurate,
    eval_itinerary_events_match_actual_events,
    eval_itinerary_satisfies_interests,
    eval_total_cost_is_within_budget,
    eval_activities_and_weather_are_compatible,
    eval_traveler_feedback_is_incorporated,  # Add this new evaluation
]

get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=ALL_EVAL_FUNCTIONS,
)

✅ Traveler Tyler has a match with interest {<Interest.COOKING: 'cooking'>} at Serve & Savor: Tennis and Taste Luncheon
✅ Traveler Tyler has a match with interest {<Interest.MUSIC: 'music'>} at Morning Groove Dance Party
✅ Traveler Tyler has a match with interest {<Interest.COOKING: 'cooking'>} at Palette & Palate: Art Meets Cooking Experience
✅ Traveler Tyler has a match with interest {<Interest.TECHNOLOGY: 'technology'>} at Tech & Film Fusion Night
✅ Traveler Annie has a match with interest {<Interest.MUSIC: 'music'>, <Interest.FITNESS: 'fitness'>} at Morning Groove Dance Party
✅ Traveler Annie has a match with interest {<Interest.ART: 'art'>} at Palette & Palate: Art Meets Cooking Experience
✅ Activity Serve & Savor: Tennis and Taste Luncheon (on 2025-06-10) and weather 'clear' are compatible.
✅ Activity Morning Groove Dance Party (on 2025-06-11) and weather 'partly cloudy' are compatible.
✅ Activity Palette & Palate: Art Meets Cooking Experience (on 2025-06-11) and weather 'partly c

EvaluationResults(success=False, failures=["Traveler feedback was not successfully incorporated into the revised travel plan. Response: NOT_INCORPORATED\nREASON: The revised travel plan includes only a single activity on June 10 and June 12, which does not fulfill the traveler's feedback to have at least two activities per day. Only one day (June 11) meets the requirement, so the feedback is not incorporated into the travel plan as specified."], eval_functions=['eval_start_end_dates_match', 'eval_total_cost_is_accurate', 'eval_itinerary_events_match_actual_events', 'eval_itinerary_satisfies_interests', 'eval_total_cost_is_within_budget', 'eval_activities_and_weather_are_compatible', 'eval_traveler_feedback_is_incorporated'])

In [23]:
# Define the ReAct system prompt for the Itinerary Revision Agent.

from utils import print_in_box

ITINERARY_REVISION_AGENT_SYSTEM_PROMPT = f"""
You are the Itinerary Revision Agent, responsible for reviewing and improving a travel itinerary for the user according to their feedback and trip requirements.

## Task

Carefully review the traveler's feedback and the current travel plan. Revise the proposed itinerary to better meet the traveler's needs, preferences, and any feedback provided. At each revision step, invoke the available evaluation tools to check if the current itinerary satisfies all constraints (dates, interests, cost, event match, weather compatibility, traveler feedback, etc). Before you provide your final output, always run the evaluation tools again on your proposed plan to ensure all requirements are satisfied.

## Available Tools

You have access to the following tools (functions):
   - calculator_tool,
   - get_activities_by_date_tool,
   - run_evals_tool,
   - final_answer_tool,

To use a tool, return a single ACTION of the following format (as one tool call per OUTPUT):
{{"tool_name": "[tool_name]", "arguments": {{"arg1": "value1", ...}}}}

## Output Format

    THOUGHT:
    Briefly explain your reasoning, outlining what you considered and your intended next step.

    ACTION:
    Output a single ACTION as a JSON object using the format:
    {{"tool_name": "[tool_name]", "arguments": {{"arg1": "value1", ...}}}}

## Context

Additional important information for you to perform your task:
- The travel plan follows the TravelPlan Pydantic schema noted here {TravelPlan.model_json_schema()}
- Feedback from the user ("Traveler Feedback") describes changes they want or problems they noticed.
- Always be sure to revisit previous steps as needed if an evaluation fails, and work towards a plan that satisfies all constraints.
- You may only return either a THOUGHT and then an ACTION in a single output; do not output the final revised travel plan until all evaluations succeed.
"""  # noqa



class ItineraryRevisionAgent(ChatAgent):
    system_prompt = ITINERARY_REVISION_AGENT_SYSTEM_PROMPT
    tools = ALL_TOOLS

    def get_observation_string(self, tool_call_obj) -> str:
        """Extracts the observation from the thought-action response."""

        if "tool_name" not in tool_call_obj:
            return "OBSERVATION: No tool name specified."

        if "arguments" not in tool_call_obj:
            return "OBSERVATION: No arguments specified."

        # If the arguments are not a dictionary, state the error
        if not isinstance(tool_call_obj["arguments"], dict):
            return f"OBSERVATION: Arguments should be a dictionary, got {type(tool_call_obj['arguments'])} instead."

        # If the tool name is not a string, state the error
        if not isinstance(tool_call_obj["tool_name"], str):
            return f"OBSERVATION: Tool name should be a string, got {type(tool_call_obj['tool_name'])} instead."

        tool_name = tool_call_obj["tool_name"]
        arguments = tool_call_obj["arguments"]

        tool_fn = None

        for tool in self.tools:
            if tool.__name__ == tool_name:
                tool_fn = tool
                break

        if tool_fn is None:
            return f"OBSERVATION: Unknown tool name '{tool_name}' in action string."

        try:
            tool_response = tool_fn(**arguments)
            return f"OBSERVATION: Tool {tool_name} called successfully with response: {tool_response}"
        except Exception as e:
            return f"OBSERVATION: Error occurred while calling tool {tool_name}: {e}"

    def run_react_cycle(
        self, original_travel_plan: TravelPlan, max_steps: int = 10, model: Optional[OpenAIModel] = None, client = None,
    ) -> TravelPlan:
        """Runs the ReAct cycle to revise the itinerary based on the evaluation results."""
        from json_repair import repair_json

        # Provide the original travel plan to revise
        self.add_message(
            role="user",
            content=f"Here is the itinerary for revision:\n{original_travel_plan.model_dump_json()}",
        )
        resp = None

        # Run the ReAct cycle for a maximum number of steps
        for step in range(max_steps):
            # Get the thought-action response from the agent
            resp = self.get_response(model=model, client=client) or ""

            # If there is no action, report it to the LLM and continue
            if "ACTION:" not in resp:
                self.add_message(role="user", content="No action found in response.")
                continue

            action_string = resp.split("ACTION:")[1].strip()

            # Parse the tool call JSON from the action string
            try:
                # Fix any JSON formatting issues. e.g. missing closing braces, etc.
                action_string = repair_json(action_string)
                tool_call_obj = json.loads(action_string)
            except json.JSONDecodeError:
                print(f"Invalid JSON in action string: {action_string}")
                self.add_message(
                    role="user",
                    content=f"Invalid JSON in action string: {action_string}",
                )
                continue

            tool_name = tool_call_obj.get("tool_name", None)

            # If the final answer tool is called, validate and return the final travel plan
            if tool_name == "final_answer_tool":
                try:
                    new_travel_plan = TravelPlan.model_validate(
                        tool_call_obj["arguments"].get("final_output", tool_call_obj["arguments"])
                    )
                    return new_travel_plan
                except Exception as e:
                    self.add_message(
                        role="user", content=f"Error validating final answer: {e}"
                    )
                    continue

            # For all other tools, execute the tool call and add the observation
            else:
                # Add the 
                observation_string = self.get_observation_string(
                    tool_call_obj=tool_call_obj
                )
                self.add_message(role="user", content=observation_string)

        raise RuntimeError(
            f"ReAct cycle did not complete within {max_steps} steps. Last response: {resp}"
        )

# Instantiate the Itinerary Revision Agent
itinerary_revision_agent = ItineraryRevisionAgent()

# Let's get a single THOUGHT/ACTION response back to check that the agent is working as expected.
resp = itinerary_revision_agent.chat(
    user_message=f"Here is the itinerary for revision: {travel_plan_1.model_dump_json(indent=2)}",
    add_to_messages=False,
    model=MODEL,
    client=client,
) or ""

print_in_box(resp, "Raw Response")
# Check for THOUGHT
if "THOUGHT:" in resp:
    print("✅ `THOUGHT:` found in raw the response, as expected.")
else:
    print("❌ Expected `THOUGHT:` in raw the response. Please check the system prompt (output format).")
# Check for ACTION
if "ACTION:" in resp:
    print("✅ `ACTION:` found in raw the response, as expected.")
else:
    print("❌ Expected `ACTION:` in raw the response. Please check the system prompt (output format).")
if "\"tool_name\"" in resp:
    print("✅ `\"tool_name\":` found in raw the response, as expected.")
else:
    print("❌ Expected `\"tool_name\":` in raw the response. Please check the system prompt (output format).")



╔══════════════════════════════════════[ ItineraryRevisionAgent - System Prompt ]══════════════════════════════════════╗
║ You are the Itinerary Revision Agent, responsible for reviewing and improving a travel itinerary for the user        ║
║ according to their feedback and trip requirements.                                                                   ║
║ ## Task                                                                                                              ║
║ Carefully review the traveler's feedback and the current travel plan. Revise the proposed itinerary to better meet   ║
║ the traveler's needs, preferences, and any feedback provided. At each revision step, invoke the available evaluation ║
║ tools to check if the current itinerary satisfies all constraints (dates, interests, cost, event match, weather      ║
║ compatibility, traveler feedback, etc). Before you provide your final output, always run the evaluation tools again  ║
║ on your proposed plan to ensu

In [24]:
# Now let's run the ReAct cycle multiple times to get the revised itinerary.
# Note: If this takes more than a few minutes, then the agent may be stuck in a loop.
# Examine the traces to understand where it is failing and see if adjusting the system prompt helps.
# Since LLMs are stochastic, you will get different results each time you run this cell.
# No changes needed here.

itinerary_revision_agent = ItineraryRevisionAgent()
travel_plan_2 = itinerary_revision_agent.run_react_cycle(
    original_travel_plan=travel_plan_1, max_steps=15,
    model=MODEL,
    client=client,
)

print("✅ Revised itinerary generated successfully. Congratulations!")



╔══════════════════════════════════════[ ItineraryRevisionAgent - System Prompt ]══════════════════════════════════════╗
║ You are the Itinerary Revision Agent, responsible for reviewing and improving a travel itinerary for the user        ║
║ according to their feedback and trip requirements.                                                                   ║
║ ## Task                                                                                                              ║
║ Carefully review the traveler's feedback and the current travel plan. Revise the proposed itinerary to better meet   ║
║ the traveler's needs, preferences, and any feedback provided. At each revision step, invoke the available evaluation ║
║ tools to check if the current itinerary satisfies all constraints (dates, interests, cost, event match, weather      ║
║ compatibility, traveler feedback, etc). Before you provide your final output, always run the evaluation tools again  ║
║ on your proposed plan to ensu

In [25]:
# Last let's double check that the revised travel plan passes all evaluation functions.
# No changes needed here.

eval_results_2 = get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_2,
    eval_functions=ALL_EVAL_FUNCTIONS,
)

assert eval_results_2.success, f"❌ Read the traces above and modify the system prompt.\n\nFailures: {eval_results_2.failures}"

print("✅ All evaluation functions passed successfully for the revised travel plan.")

eval_results_2

✅ Traveler Tyler has a match with interest {<Interest.TECHNOLOGY: 'technology'>} at FutureTech Breakfast Meet-Up
✅ Traveler Tyler has a match with interest {<Interest.COOKING: 'cooking'>} at Serve & Savor: Tennis and Taste Luncheon
✅ Traveler Tyler has a match with interest {<Interest.MUSIC: 'music'>} at Morning Groove Dance Party
✅ Traveler Tyler has a match with interest {<Interest.COOKING: 'cooking'>} at Palette & Palate: Art Meets Cooking Experience
✅ Traveler Tyler has a match with interest {<Interest.MUSIC: 'music'>} at Soundtrack Picnic: Lunchtime Movies & Melodies
✅ Traveler Tyler has a match with interest {<Interest.TECHNOLOGY: 'technology'>} at Tech & Film Fusion Night
✅ Traveler Annie has a match with interest {<Interest.MUSIC: 'music'>, <Interest.FITNESS: 'fitness'>} at Morning Groove Dance Party
✅ Traveler Annie has a match with interest {<Interest.ART: 'art'>} at Palette & Palate: Art Meets Cooking Experience
✅ Traveler Annie has a match with interest {<Interest.MUSIC: 'm

EvaluationResults(success=True, failures=[], eval_functions=['eval_start_end_dates_match', 'eval_total_cost_is_accurate', 'eval_itinerary_events_match_actual_events', 'eval_itinerary_satisfies_interests', 'eval_total_cost_is_within_budget', 'eval_activities_and_weather_are_compatible', 'eval_traveler_feedback_is_incorporated'])

In [26]:
# Show the final travel plan in a readable format.
# No changes needed here.

from IPython.display import display

for itinerary_day in travel_plan_2.itinerary_days:
    print(f"Date: {itinerary_day.date}")
    print(
        f"Weather: {itinerary_day.weather.condition} ({itinerary_day.weather.temperature}°{itinerary_day.weather.temperature_unit})"
    )

    activities_df = pd.DataFrame(
        [
            activity_recommendation.activity.model_dump()
            for activity_recommendation in itinerary_day.activity_recommendations
        ]
    )
    display(activities_df)

Date: 2025-06-10
Weather: clear (31.0°celsius)


,activity_id,name,start_time,end_time,location,description,price,related_interests
0,event-2025-06-10-0,FutureTech Breakfast Meet-Up,2025-06-10 09:00:00,2025-06-10 11:00:00,"The Innovation Atrium, Tech District, AgentsVille","Join fellow technology enthusiasts for a dynamic morning at the FutureTech Breakfast Meet-Up! Dive into the latest trends in tech, gadget demos, and networking opportunities over coffee and fresh pastries. Held indoors at the spacious Innovation Atrium, this event is perfect for tech lovers eager to exchange ideas and discover new possibilities in a comfortable, modern setting.",20,[Interest.TECHNOLOGY]
1,event-2025-06-10-1,Serve & Savor: Tennis and Taste Luncheon,2025-06-10 12:00:00,2025-06-10 13:30:00,"The Grand Racquet Terrace, AgentsVille","Join us for 'Serve & Savor,' the ultimate crossover event for cooking and tennis enthusiasts in AgentsVille! Kick off your lunch hour with a friendly round of doubles on our outdoor courts, then unwind with a hands-on cooking workshop led by a local chef, where you'll prepare and enjoy delicious energy-boosting recipes. Whether you come for the sport or the flavors, this energizing luncheon celebrates both passions in a lively outdoor setting. Ideal for anyone who loves to play, cook, or simply savor fresh food and fun!",20,"[Interest.COOKING, Interest.TENNIS]"


Date: 2025-06-11
Weather: partly cloudy (34.0°celsius)


,activity_id,name,start_time,end_time,location,description,price,related_interests
0,event-2025-06-11-0,Morning Groove Dance Party,2025-06-11 09:00:00,2025-06-11 10:30:00,"Rhythm Hall, Center Plaza, AgentsVille","Start your day with energy and joy at the Morning Groove Dance Party! This lively event welcomes dancers of all levels to join a vibrant indoor session filled with upbeat music and fun routines. Whether you love modern pop, Latin beats, or classic disco, our dance instructors will guide you to move and groove. Connect with fellow dance lovers in the colorful atmosphere of Rhythm Hall. Perfect for fans of dancing, music, and fitness. Let the rhythm move you! (Indoor event.)",15,"[Interest.DANCING, Interest.MUSIC, Interest.FITNESS]"
1,event-2025-06-11-3,Palette & Palate: Art Meets Cooking Experience,2025-06-11 18:30:00,2025-06-11 20:30:00,"The Creative Canvas Studio, Artisanal Lane, AgentsVille","Immerse yourself in a colorful evening where art and cooking blend together! At 'Palette & Palate,' participants will begin indoors at The Creative Canvas Studio with a guided session to paint their own culinary-inspired masterpiece. Afterwards, a local chef will lead an interactive cooking class, teaching you how to craft vibrant, edible works of art. Whether you're an art enthusiast, a food lover, or both, this creative night is perfect for socializing and expressing yourself through color and flavor! All materials and ingredients are provided. This event is held indoors and welcomes all experience levels in art and cooking.",25,"[Interest.ART, Interest.COOKING]"


Date: 2025-06-12
Weather: thunderstorm (28.0°celsius)


,activity_id,name,start_time,end_time,location,description,price,related_interests
0,event-2025-06-12-1,Soundtrack Picnic: Lunchtime Movies & Melodies,2025-06-12 12:00:00,2025-06-12 13:30:00,"Starlight Amphitheater, AgentsVille","Experience the magic of classic movie scenes paired with live music at the outdoor Starlight Amphitheater! Bring your lunch and relax on the lawn as musicians perform iconic film soundtracks while selected clips light up our open-air screen. Perfect for movie buffs and music lovers alike, this engaging event celebrates both arts in a sunny lunchtime setting. In case of rain, the event will move indoors to the adjacent Harmony Hall. Come for the tunes, stay for the cinematic wonder!",15,"[Interest.MOVIES, Interest.MUSIC]"
1,event-2025-06-12-3,Tech & Film Fusion Night,2025-06-12 19:00:00,2025-06-12 21:30:00,"Virtual Reality Theater, Silicon Plaza, AgentsVille","Dive into an immersive evening where the magic of movies meets the latest in technology! Join fellow movie buffs and tech enthusiasts for a special screening of cutting-edge sci-fi short films, followed by an interactive panel with local filmmakers and VR technologists. Experience the future of entertainment and discuss how technology is transforming the world of cinema. This exciting, indoor event at the Virtual Reality Theater is perfect for anyone interested in technology and movies.",15,"[Interest.TECHNOLOGY, Interest.MOVIES]"


## And, just for fun!

In [28]:
# And finally, just for fun, let's narrate the trip.
# No changes needed here.

from utils import narrate_my_trip

narrate_my_trip(
    vacation_info=vacation_info,
    itinerary=travel_plan_2,
    client=client,
    model=MODEL,  # Optionally, you can change the model here
)


You’re set for an exciting trip to AgentsVille from June 10th to June 12th, 2025, with two travelers: Tyler, age 33, who enjoys music, cooking, comedy, and technology; and Annie, age 25, who is interested in fitness, music, theatre, and art. Your budget for the trip is 130, and the total planned activities cost 110, leaving you some flexibility. The itinerary thoughtfully blends your varied interests, balancing indoor and outdoor events while considering the local weather.

### June 10th – A Clear and Bright Start  
The day begins with the **FutureTech Breakfast Meet-Up** in the Innovation Atrium of the Tech District. This indoor morning event specializes in the latest developments in technology, gadget demos, and networking over coffee and pastries—perfect for Tyler’s passion for tech. Later, as the weather is clear, you’ll enjoy the **Serve & Savor: Tennis and Taste Luncheon** at the Grand Racquet Terrace. This unique crossover event offers a friendly tennis session paired with a hands-on cooking workshop led by a local chef, making it ideal for both Tyler’s love of cooking and Annie’s fitness interests.

### June 11th – Energetic and Creative Experiences  
With partly cloudy skies, your day features two standout indoor activities. Start with the **Morning Groove Dance Party** at Rhythm Hall, where upbeat music and dance routines cater nicely to Annie’s interests in fitness and music. It promises an energetic, social start to the day that will lift your spirits. In the evening, dive into **Palette & Palate: Art Meets Cooking Experience** at The Creative Canvas Studio. This event artfully combines Annie’s love of art with Tyler’s cooking interest through an interactive painting and cooking class. It’s a perfect opportunity to get creative, socialize, and enjoy a relaxed indoor atmosphere.

### June 12th – A Rainy Day with Engaging Indoors  
Expect thunderstorms, so your activities are suited for staying dry while still enjoying rich cultural experiences. At midday, the **Soundtrack Picnic: Lunchtime Movies & Melodies** offers an outdoor movie and live music event that will move indoors if the rain comes. It’s an inviting way to experience classic film soundtracks alongside live performances, fitting well with both your music and movie interests. To close your trip, the **Tech & Film Fusion Night** at the Virtual Reality Theater blends Tyler’s love for technology and movies with an immersive evening of sci-fi screenings and interactive discussions with filmmakers and VR experts. This indoor event is perfect for a memorable and stimulating finale.

Overall, this itinerary provides a thoughtful balance of your interests across music, cooking, technology, fitness, art, and movies, all while adapting to the weather and ensuring each day offers enjoyable and enriching experiences.

In [31]:
from IPython.display import Audio, display
filename="/tmp/my_trip_narration.mp3"
display(Audio(filename=filename))

ValueError: rate must be specified when data is a numpy array or list of audio samples.

In [ ]:

get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_itinerary_satisfies_interests],
)


✅ Traveler Tyler has a match with interest {<Interest.TECHNOLOGY: 'technology'>} at FutureTech Breakfast Meet-Up
✅ Traveler Tyler has a match with interest {<Interest.MUSIC: 'music'>} at Morning Groove Dance Party
✅ Traveler Tyler has a match with interest {<Interest.COOKING: 'cooking'>} at Palette & Palate: Art Meets Cooking Experience
✅ Traveler Tyler has a match with interest {<Interest.TECHNOLOGY: 'technology'>} at Tech & Film Fusion Night
✅ Traveler Annie has a match with interest {<Interest.FITNESS: 'fitness'>, <Interest.MUSIC: 'music'>} at Morning Groove Dance Party
✅ Traveler Annie has a match with interest {<Interest.ART: 'art'>} at Palette & Palate: Art Meets Cooking Experience


EvaluationResults(success=True, failures=[], eval_functions=['eval_itinerary_satisfies_interests'])

## CONGRATULATIONS! 🎉🥳👏

You have successfully planned a stellar vacation to AgentsVille! Your AI travel agent has demonstrated advanced reasoning techniques, including role-based prompting, chain-of-thought reasoning, ReAct prompting, and feedback loops

Give yourself a pat on the back for completing this project and completing this course!